In [2]:
# --- Imports and Environment Setup ---
from dotenv import load_dotenv
load_dotenv() # Load environment variables from .env file

import logging

from pydantic import Field, BaseModel # For data validation and settings management
from typing import List, Literal # For type hinting

from langgraph.graph import MessagesState, StateGraph, START, END # Core LangGraph components for building stateful graphs
from langgraph.types import Command, interrupt # For controlling graph flow, e.g., human-in-the-loop

from langchain_core.prompts import PromptTemplate # For creating flexible prompts for LLMs
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage # For structuring messages in conversations
from langchain_google_genai import ChatGoogleGenerativeAI # Google Generative AI model wrapper

from experiments.gemini_phase1_toolgen import tool_compile_graph

print("imports complete")

# --- Logging Configuration ---
# Set up the logger for application-wide logging
logging.basicConfig(
    level=logging.INFO,  # Set to DEBUG for detailed logs, INFO for general information
    format="%(asctime)s - %(levelname)s - %(message)s", # Log message format
    handlers=[
        # logging.FileHandler("scraper.log"),  # Option to log to a file (currently commented out)
        logging.StreamHandler()  # Log to console
    ]
)
logger = logging.getLogger(__name__) # Get a logger instance for the current module

# --- LLM Initialization ---
# Initialize the Language Model (LLM) to be used throughout the application
# Using Google's Gemini Flash model with a temperature of 0 for deterministic outputs
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-05-20", temperature=0)


# --- Agent Requirement Analysis ---
# This section defines the process for understanding user requirements for building an AI agent.

# Prompt template for the Requirement Analysis LLM
REQ_ANALYSIS_PROMPT = """Your job is to get information from a user about what kind of agent they wish to build.

You should get the following information from them:

- What the objective of the agent is
- Various usecases of the agent
- Some examples of what the agent will be doing (Input and expected output pairs)

If you are not able to discern this info, ask them to clarify, you can suggest and see if user confirms the suggestions.

After you are able to discern all the information, call the tool AgentInstruction"""

class AgentInstructions(BaseModel):
    """
    Pydantic model to structure the instructions for building the Agent.
    This model is used as a tool for the LLM to output structured information.
    """
    objective: str = Field(description="What is the primary objective of the agent")
    usecases: List[str] = Field(description="What are the various responsibilities of the agent which it needs to fulfill")
    examples: str = Field(description="What are some examples of the usage of the agent (input query and expected output from the agent) ?")

class AgentBuilderState(MessagesState):
    """
    Represents the state of the main agent building graph.
    Inherits from MessagesState to include a list of messages.
    """
    agent_instructions: AgentInstructions = Field(description="The requirement analysis generated by the model.")
    python_code: str = Field(description="The Python code generated for the agent")

def requirement_analysis_node(state: AgentBuilderState) -> Command[Literal["requirement_analysis_node", "code_node"]]:
    """
    LangGraph node for performing requirement analysis.
    It interacts with the LLM to gather agent specifications from the user.
    If information is insufficient, it interrupts the graph for user input.
    Otherwise, it proceeds to the code generation node.
    """
    logger.info("Executing requirement_analysis_node")
    llm_with_tool = llm.bind_tools([AgentInstructions]) # Bind the AgentInstructions Pydantic model as a tool
    
    # Invoke the LLM with the system prompt and current message history
    response = llm_with_tool.invoke([SystemMessage(content=REQ_ANALYSIS_PROMPT)] + state["messages"])
    
    if not response.tool_calls:
        logger.info("LLM requires more information or is in conversation.")
        value = interrupt(response.content)
        return Command(goto="requirement_analysis_node", update={"messages": [response, HumanMessage(content=value)]})
        
    logger.info("LLM successfully called AgentInstructions tool.")
    agent_instructions_args = response.tool_calls[0]["args"]
    agent_instructions = AgentInstructions(**agent_instructions_args)
    
    return Command(
        goto="code_node",
        update={"messages": [response], "agent_instructions": agent_instructions}
    )

# --- Agent Code Generation ---
# This section focuses on generating the main Python code for the agent based on the gathered requirements.

CODE_GEN_PROMPT = PromptTemplate.from_template("""
You are an expert Python programmer specializing in AI agent development via the Langgraph and Langchain SDK . Your primary task is to generate compilable, logical, and complete Python code for a LangGraph state graph based on user input below. You must prioritize LLM-based implementations for relevant tasks and consider advanced graph architectures.

**Input:**
<INPUT>
<OBJECTIVE>
{objective}
</OBJECTIVE>
<USECASES>
{usecases}
</USECASES>
<EXAMPLES>
{examples}
</EXAMPLES>
</INPUT>
---
**Phase 1: Evaluating best architecture for the given INPUT**

1.  **Identify Potential Architectures:** Consider if the described INPUT aligns with or would benefit from known advanced LangGraph architectures such as:
    * **Plan and Execute**: Does the INPUT imply an agent which might need a planning step (e.g., breaking down a complex task) followed by the execution of those plans by one or more action nodes?
    * **Agent Supervisor / Hierarchical Agent Teams**: Is the INPUT best served by a supervisor agent dispatching tasks to specialized worker agents, or a hierarchy of agents making decisions and delegating?
    * **Multi-Agent Collaboration (e.g., Swarm Architecture)**: Does the problem benefit from multiple agents working in parallel or collaboratively, perhaps sharing insights or contributing to a common goal?
    * **Reflection / Self-Correction (e.g., Self-Discover frameworks)**: Are there indications of iterative refinement, where results are evaluated and the process is adjusted?
    * **Human in the Loop (HITL)**: Does the `description` of any node, or the overall process, imply a need for human review, approval, correction, or explicit input at specific stages (e.g., before executing a critical action, when confidence is low, or for subjective assessments)?

2.  **Architectural Decision:**
    * If you determine that one or more of these architectures are strongly applicable to the INPUT, choose to implement it.
    * If no specific advanced architecture seems directly applicable for the given INPUT, proceed with a standard stateful graph construction based on the explicit langgraph nodes and edges.

3.  **Initial Comment:** At the very beginning of your generated Python script, include a comment block stating:
    * Which LangGraph architecture(s) (if any) you've identified and chosen to implement, with a brief justification based on your interpretation of the INPUT, provide dry runs of the usecases/examples.
    * If you are proceeding with a standard graph, mention that.

4. Generate a JSON representation of the architecture you have chosen, including:
a.  `nodes`: A dictionary where each key is a unique node ID. The value for each node ID is an object containing:
    * `id`: The node's identifier.
    * `schema_info`: A string describing the structure of the `GraphState` (e.g., "GraphState:\\n type: TypedDict\\n fields:\\n - name: input\\n type: str..."). You will need to parse this to define the `GraphState` TypedDict.
    * `input_schema`: The expected input schema for the node (typically "GraphState").
    * `output_schema`: The schema of the output produced by the node (typically "GraphState", indicating a partial update).
    * `description`: A natural language description of what the node does. This is crucial for determining implementation strategy and overall architecture.
    * `function_name`: The suggested Python function name for this node.
    * `code` (optional): A string containing Python code for the node's function. **Treat this `code` primarily as an illustration or a very basic version. Prioritize LLM-based solutions if the `description` suggests a more robust approach is needed.**

b.  `edges`: A list of objects, each describing a directed edge in the graph. Each edge object contains:
    * `source`: The ID of the source node (or "__START__" for the graph's entry point).
    * `target`: The ID of the target node (or "__END__" for a graph termination point).
    * `routing_conditions`: A natural language description of the condition under which this edge is taken, especially for conditional edges.
    * `conditional`: A boolean flag, `true` if the edge is part of a conditional branch, `false` otherwise.


---

** Phase 2: Graph Creation
**Phase 2: Python Code Generation**

Generate a single, self-contained, and compilable Python script that implements your chosen strategy.

1.  **Imports:** Include all necessary Python libraries (e.g., `typing`, `langgraph.graph`, `langgraph.checkpoint.memory`, LLM client libraries like `langchain_openai`, `langchain_google_genai`, `langchain_core.pydantic_v1`, `langchain_core.tools`, `re`).

2.  **State Definition (`GraphState`):**
    * Define a `GraphState` class using `MessagesState` (langgraph prebuilt class).

3.  **Node Implementation (Python Functions):**
    For each conceptual node in your chosen architecture (these may map directly to JSON you define):
    * Create a Python function. This function must accept the `GraphState` and return a dictionary representing the partial update to the state.
    * **Decision Logic for Implementation (Prioritize LLM, No Mock Data):**
        * **Default to LLM-Based Solutions:** Your default stance should be to implement an **LLM-based solution** if the node's `description` (from JSON or your architectural design) suggests tasks like:
            * Natural Language Understanding (NLU)
            * Complex classification or routing
            * Content generation or summarization
            * Tool selection and usage
            * Planning or complex decision-making.
            * Any task where an LLM would provide more robust, flexible, or intelligent behavior than simple hardcoded logic.
        * **Handling Provided `code`:** If `code` is present in the JSON for a node, treat it as a **low-priority hint or a simplistic example**. Do **not** simply copy it if an LLM approach is more appropriate for the described task.
        * **Algorithmic Logic (Use Sparingly):** Only use purely algorithmic Python code (like from the `code` attribute or written new) if the node's task is genuinely simple, deterministic (e.g., basic data formatting, fixed calculation), *and* an LLM would offer no significant benefit for that specific, narrow function.
        * **Functional LLM Calls:** When an LLM is used, instantiate a generic model (e.g., `llm = ChatOpenAI(model="gpt-3.5-turbo")` or `llm = ChatGoogleGenerativeAI(model="gemini-pro")`) and include a **functional, descriptive prompt** relevant to the node's task. Ensure the code for the LLM call is complete and not just a comment. Add a `TODO` comment for the user to specify API keys and potentially refine the model/prompt.
        * **No Mock Data:** Generated functions must be logical and aim for completeness. **Avoid using mock data or overly simplistic placeholder logic** where an LLM or a proper algorithmic implementation is expected.
        * **Structured Output & Tools:** If the task implies structured output from an LLM or the use of tools, define necessary Pydantic models and/or LangChain tools, and integrate them with the LLM call.
            * Define a Pydantic model (e.g., `from langchain_core.pydantic_v1 import BaseModel, Field`) representing the desired structured output.
            * If implementing an LLM call, configure it to use the Pydantic model for its output (e.g., with OpenAI's function calling/tool usage features, or by instructing the LLM to generate JSON conforming to the model).
        * **Tool Definition and Usage:** If a node's `description` (or your architectural design) implies the LLM within that node needs to interact with external systems, perform specific actions, or fetch data (e.g., "search customer database," "get weather update"):
                * Define these capabilities as discrete LangChain tools using the `@tool` decorator (e.g., `from langchain_core.tools import tool`).
                * **Crucially, each tool's internal Python function should be self-contained and directly perform its advertised action** (e.g., make a specific API call to an external service, run a local script, perform a calculation, retrieve data algorithmically). **Avoid embedding a *new, separate general-purpose LLM call within the tool's own implementation logic* unless the tool's explicit and documented purpose is to be a specialized, self-contained sub-agent (which is an advanced case).** The primary LLM within the graph node is responsible for *deciding to call* the tool and for interpreting its output.
                * Bind these well-defined tools to the LLM instance operating within that graph node. The node's LLM will then intelligently decide when to call a tool and with what inputs.
        * **Human in the Loop Nodes:** If you've designed a HITL step as a dedicated node, its function might primarily format data for human review and then process the subsequent human input (which would be added to the state, potentially by an external mechanism or a subsequent node). The graph might pause using an interruption mechanism tied to this node.
        * **State Coherence:** Ensure variable assignments and updates within node functions are coherent with the `GraphState` definition and how state is managed in LangGraph.

4.  **Graph Construction (`StatefulGraph`):**
    * Instantiate `StatefulGraph(GraphState)`.
    * Add each implemented node function to the graph using `graph.add_node("node_id", node_function)`.
    * Set the graph's entry point using `graph.add_edge(START, "entry_node_id")` where `"entry_node_id"` is the target of the edge originating from `"__START__"`.

5.  **Edge Implementation:**
    * Iterate through the `edges` list in the JSON.
    * **Regular Edges:** If `conditional` is `false`:
        * If `target` is `__END__`, use `graph.add_edge(source_node_id, END)`.
        * Otherwise, use `graph.add_edge(source_node_id, target_node_id)`.
    * **Conditional Edges:** If `conditional` is `true`:
        * The `source` node of these conditional edges is expected to produce some output in the `GraphState` (e.g., an `intent` field) that determines the next path.
        * Create a separate routing function (e.g., `def route_after_source_node(state: GraphState) -> str:`).
        * This routing function must inspect the relevant fields in the `state` and return the string ID of the next node to execute, based on the logic described in the `routing_conditions` for each conditional edge originating from that source.
        * Use `graph.add_conditional_edges(source_node_id, routing_function, {{ "target_id_1": "target_id_1", "target_id_2": "target_id_2", ... "__END__": END }})`. The keys in the dictionary are the possible return values from your routing function, and the values are the actual node IDs or `END`.

6.  **Compilation:**
    * Instantiate an `InMemoryCheckpointer`: `checkpointer = InMemoryCheckpointer()`.
    * Compile the graph: `final_app = graph.compile(checkpointer=checkpointer)`. The compiled graph must be assigned to a variable named `final_app`.

---
**Phase 3: Required Keys/Credentials Identification**

After generating the complete Python script, add a separate section at the end of your response, clearly titled:
`## Required Keys and Credentials`

In this section, list all environment variables or API keys a user would need to set for the generated code to execute successfully (e.g., `OPENAI_API_KEY`, `GOOGLE_API_KEY`, tool-specific keys). If no external keys are needed, state that.

---
**Important Considerations (General):**
* The primary goal is **compilable, logical, and functionally plausible Python code** that intelligently interprets the JSON input.
* Focus on creating a system that leverages LLMs effectively for tasks suited to them.
* Ensure node functions correctly update and return relevant parts of the `GraphState`.
* If the provided `code` in the JSON uses specific libraries (e.g., `re`), make sure the corresponding import is included at the top of the script.
* Handle `__START__` and `__END__` correctly in edge definitions. `langgraph.graph.START` and `langgraph.graph.END` should be used.

Please generate the Python code and the list of required keys now:
""")

def code_node(state: AgentBuilderState):
    """
    LangGraph node to generate the final Python code for the agent.
    It uses the gathered agent_instructions and the CODE_GEN_PROMPT.
    """
    logger.info("Executing code_node")
    instructions: AgentInstructions = state["agent_instructions"]
    
    # Invoke LLM to generate code based on the detailed prompt and instructions
    code_output = llm.invoke([HumanMessage(content=CODE_GEN_PROMPT.format(
        objective=instructions.objective,
        usecases=instructions.usecases,
        examples=instructions.examples
    ))])
    
    logger.info("Python code generated by LLM.")
    # Return the generated Python code and an AI message
    return {
        "messages": [AIMessage(content="Generated final python code!")],
        "python_code": code_output.content,
    }


# --- Main Agent Generation Graph ---
# This is the primary graph that orchestrates the entire agent building process.


# --- Example Invocation (Optional) ---
# This section can be used to demonstrate how to run the main graph.
# For example:
# if __name__ == "__main__":
#     logger.info("Starting agent generation process...")
#     initial_input = {"messages": [HumanMessage(content="I want to build an agent that can tell me the weather.")]}
#     config = {"configurable": {"thread_id": "user-thread-1"}}
    
#     for event in agent_generator_graph.stream(initial_input, config=config):
#         for key, value in event.items():
#             logger.info(f"Event from graph: {key} - {value}")
#             if key == "requirement_analysis" and isinstance(value, dict) and 'messages' in value:
#                 last_message = value['messages'][-1]
#                 if isinstance(last_message, HumanMessage) and last_message.content.startswith("__interrupt__"):
#                     # This is a simplified way to handle interruption for demo.
#                     # In a real app, you'd present this to the user and get their input.
#                     user_response = input(f"Agent asks: {last_message.content[len('__interrupt__'):].strip()} Your response: ")
#                     # How to reinvoke or continue with new input needs careful handling with stream/invoke
#                     # For simplicity, this example doesn't fully implement the interactive loop here.
#                     logger.info(f"User input received: {user_response} (manual continuation needed for stream)")


#     final_state = agent_generator_graph.get_state(config)
#     logger.info("\n--- Final Generated Python Code ---")
#     print(final_state.values.get("python_code"))
#     logger.info("Agent generation process complete.")

imports complete


In [3]:
analysis_compile_prompt = """
You are an expert LangGraph code refactoring AI. Your task is to analyze the provided Python code for a LangGraph implementation and automatically correct it to ensure adherence to best practices, specifically concerning tool definition within nodes and the consistency of state objects.

Please perform the following analysis and apply corrections directly to the code:

Tool Definition and Usage in Nodes:

Correct Invocation: Identify and fix any instances where tools called within graph nodes are not defined or invoked correctly according to LangGraph's protocols.
Schema Adherence: Ensure that the inputs provided to tools and the outputs received from them strictly adhere to their defined schemas. Modify the code to align with these schemas if discrepancies are found.
Tool Registration (if applicable): If tools are not correctly registered or made available to the nodes that intend to use them, update the code to ensure proper registration.
Error Handling: Implement or improve error handling for tool execution failures within the nodes, making the graph more robust.
State Object Management and Consistency:

State Definition Review (and potential refinement): While the primary goal is to ensure consistency with the existing definition, if the state definition itself is unclear or problematic for achieving consistency, you may suggest minor refactorings to the state definition (clearly noting these changes).
Node-State Interaction: For each node:
Input State: Correct any instances where the node incorrectly accesses or misinterprets information from the input state.
Output State: Rectify how the node updates the state object to ensure it's consistent with its defined purpose, the overall graph flow, and the state definition. Ensure all modifications are explicit and correct.
Type Consistency: Enforce that the data types of values being read from and written to the state are consistent across different nodes and with the state definition. Apply necessary type conversions or corrections.
Immutability (where applicable): If parts of the state are intended to be immutable but are modified, adjust the node logic to respect this or ensure modifications are handled through proper state update mechanisms.
State Transitions: Refine the logic of state changes between nodes if it leads to inconsistencies or deviates from the intended graph objective.
Overall Code Health (related to tools and state):

Clarity and Readability: Refactor code related to tool usage and state manipulation to improve its clarity and readability, potentially by adding comments or restructuring logic.
Modularity: If tool definitions or state interactions can be better encapsulated within their respective nodes for improved modularity, make these changes.
Input:
You will be provided with the Python code for the LangGraph implementation.

Output:
Provide the updated and corrected LangGraph Python code.
input code:
<input_code>
{compiled_code}
</input_code>
"""
def dfs_analysis_node(state: AgentBuilderState): # Renamed for clarity
    """
    LangGraph node to analyse the code
    """
    main_agent_code = state['python_code']
    
    # Use LLM to merge the main agent code with the generated tool definitions
    response = llm.invoke([HumanMessage(content=analysis_compile_prompt.format(
        compiled_code=main_agent_code,
    ))])
    
    logger.info("Main agent code updated with fixes.")
    # The response from this LLM call is expected to be the final, complete Python code
    return {
        "messages": [AIMessage(content=response.content)], # Storing the LLM's final code as a message for now
        "python_code": response.content # Update compiled_code with the final merged code
    }


In [ ]:


from langchain_core.language_models.chat_models import BaseChatModel



In [5]:
SYSTEM_PROMPT = """
You are an expert software engineer.

You will be given a langgraph code. You need to fix it and make it runnable.
If you are not aware about some piece of code or a prebuilt function,use the get_langgraph_docs_index tool to get an index of the LangGraph docs first,
then follow up with the get_request tool. Be persistent - if your first page does
not result in confident information, keep digging!

Make sure it is correct, complete, and executable without modification.
Make sure that any generated code is contained in a properly formatted markdown code block.

You can use the following URLs with your "get_langgraph_docs_content" tool to help answer questions:

{langgraph_llms_txt}
"""

@tool
def get_langgraph_docs_content(url: str) -> str:
    """Sends a get request to a webpage and returns plain text
    extracted via BeautifulSoup."""
    res = requests.get(url).text
    soup = BeautifulSoup(res, features="html.parser")
    return soup.get_text()

def create_base_agent(model):
    langgraph_llms_txt = requests.get(
        "https://langchain-ai.github.io/langgraph/llms.txt"
    ).text
    return create_react_agent(
        model=model,
        tools=[get_langgraph_docs_content],
        prompt=SYSTEM_PROMPT.format(langgraph_llms_txt=langgraph_llms_txt),
    ).with_config(run_name="Base Agent")

def create_judge_graph(sandbox: Sandbox):
    def run_reflection(state: dict) -> dict | None:
        evaluator = create_e2b_pyright_evaluator(
            sandbox=sandbox,
            code_extraction_strategy="markdown_code_blocks",
        )

        result = evaluator(outputs=state)

        code_extraction_failed = result["metadata"] and result["metadata"].get(
            "code_extraction_failed"
        )

        if not result["score"] and not code_extraction_failed:
            return {
                "messages": [
                    {
                        "role": "user",
                        "content": f"I ran pyright and found some problems with the code you generated: {result['comment']}\n\n"
                        "Try to fix it. Make sure to regenerate the entire code snippet. "
                        "If you are not sure what is wrong, search for more information by pulling more information "
                        "from the LangGraph docs.",
                    }
                ]
            }

    return (
        StateGraph(MessagesState)
        .add_node("run_reflection", run_reflection)
        .add_edge("__start__", "run_reflection")
        .compile()
    ).with_config(run_name="Judge Agent")

_GLOBAL_SANDBOX = None

def get_or_create_sandbox():
    global _GLOBAL_SANDBOX
    if _GLOBAL_SANDBOX is None:
        _GLOBAL_SANDBOX = Sandbox("OpenEvalsPython")
    return _GLOBAL_SANDBOX

def create_reflection_agent(config: RunnableConfig = None):
    if config is None:
        config = {}
    configurable = config.get("configurable", {})
    sandbox = configurable.get("sandbox", None)
    model = configurable.get("model", None)
    if sandbox is None:
        sandbox = get_or_create_sandbox()
    if model is None:
        model = init_chat_model(
            model="gpt-4o-mini",
            max_tokens=4096,
        )
    judge = create_judge_graph(sandbox)
    return (
        create_reflection_graph(create_base_agent(model), judge, MessagesState)
        .compile()
        .with_config(run_name="Mini Chat LangChain")
    )

def code_reflection_node_updated(state: AgentBuilderState):
    """
    LangGraph node to run code reflection and fixing using E2B sandbox - updated for AgentBuilderState
    """
    logger.info("Executing code_reflection_node to test and fix code in E2B sandbox.")
    python_code = state['python_code']
    
    # Create reflection agent
    reflection_agent = create_reflection_agent({})
    
    # Run the reflection agent with the generated code
    result = reflection_agent.invoke({
        "messages": [HumanMessage(content=f"Please review and fix this LangGraph code to make it compilable and runnable:\n\n```python\n{python_code}\n```")]
    })
    
    # Extract the final message content as the fixed code
    final_message = result["messages"][-1]
    fixed_code = final_message.content if hasattr(final_message, 'content') else str(final_message)
    
    logger.info("Code reflection and fixing completed.")
    return {
        "messages": [AIMessage(content="Code has been tested and fixed using E2B sandbox reflection.")],
        "python_code": fixed_code
    }


In [10]:
main_workflow = StateGraph(AgentBuilderState)

# Add all nodes to the main workflow
main_workflow.add_node("requirement_analysis_node", requirement_analysis_node)
main_workflow.add_node("code_node", code_node)
main_workflow.add_node("tool_subgraph_processing", tool_compile_graph)
main_workflow.add_node("dfs_analysis_node", dfs_analysis_node)
main_workflow.add_node("code_reflection_node", code_reflection_node_updated)

# Define the updated workflow edges
main_workflow.add_edge(START, "requirement_analysis_node")
main_workflow.add_edge("requirement_analysis_node", "code_node")
main_workflow.add_edge("code_node", "tool_subgraph_processing")
main_workflow.add_edge("tool_subgraph_processing", "dfs_analysis_node")
main_workflow.add_edge("dfs_analysis_node", "code_reflection_node")
main_workflow.add_edge("code_reflection_node", END)
agent_generator_graph = main_workflow.compile()
logger.info("Main agent generator graph compiled.")

2025-06-05 22:08:41,435 - INFO - Main agent generator graph compiled.


In [12]:
# Run the graph until the interrupt is hit.
query= " need to create a worklow with the objective of managing my social media.  It should be able to tell me trends from social media for sports, get me all the relevant people I should contant for a spsonsoring a specific post,  suggest me content I should be posting, make content for me if I give it a description * Identifying social media trends in sports. *   Finding relevant people for sponsoring specific posts. *   Suggesting content to post. *   Generating content based on a description.  Examples: Given I ask it about trends in football, it goes through recent viral reels in football and tell me Q&A reels are trending If I tell it I want people who would be interested in a post about UCL football, it gives me current players like Dembele etc If I tell it I want to make a post about UCL football, it goes through what is trending and an agent that thinks about social media posts and tells me we should post a highlight reel"
config = {"configurable": {"thread_id": "some_id"}}
result = agent_generator_graph.invoke({"messages": HumanMessage(content=query)}, config=config) 




2025-06-05 22:08:51,151 - INFO - Executing requirement_analysis_node
2025-06-05 22:08:53,210 - INFO - LLM successfully called AgentInstructions tool.
2025-06-05 22:08:53,218 - INFO - Executing code_node
2025-06-05 22:10:27,781 - INFO - Python code generated by LLM.
2025-06-05 22:10:27,798 - INFO - Executing graph_map_step to identify and generate tool implementations.
2025-06-05 22:10:33,081 - INFO - Generating implementation for tool: search_social_media - Description: Searches social media platforms for trending topic...
2025-06-05 22:10:33,084 - INFO - Generating implementation for tool: search_influencers - Description: Searches for influencers or relevant individuals i...
2025-06-05 22:10:33,095 - INFO - Executing functional_analysis_node for: Searches social media platforms for trending topic...
2025-06-05 22:10:33,119 - INFO - Executing functional_analysis_node for: Searches for influencers or relevant individuals i...
2025-06-05 22:10:36,802 - INFO - Functional analysis complet

TimeoutException: The sandbox was not found: This error is likely due to sandbox timeout. You can modify the sandbox timeout by passing 'timeoutMs' when starting the sandbox or calling '.setTimeout' on the sandbox with the desired timeout.

In [ ]:
# Checkpointer for the main agent generation graph
main_workflow = StateGraph(AgentBuilderState) # Define state type

# Add nodes to the main workflow
main_workflow.add_node("requirement_analysis_node", requirement_analysis_node)
main_workflow.add_node("code_node", code_node)
main_workflow.add_node("tool_subgraph_processing", tool_compile_graph) # Renamed node

# Define edges for the main workflow
main_workflow.add_edge(START, "requirement_analysis_node")             # Start with requirement analysis
main_workflow.add_edge("requirement_analysis_node", "code_node")        # Then generate initial code
main_workflow.add_edge("code_node", "tool_subgraph_processing")    # Then process/implement tools via sub-graph
main_workflow.add_edge("tool_subgraph_processing", END)            # End after tool processing

# Compile the main agent generation graph
agent_generator_graph = main_workflow.compile()
logger.info("Main agent generator graph compiled.")